In [1]:
import util
import numpy as np
import paraPropPython as ppp
from paraPropPython import receiver
import scipy.signal as signal
import util
import h5py

fname = "simul_data.h5"
input_hdf = h5py.File(fname, 'r')

iceDepth = input_hdf.attrs["iceDepth"]
iceLength = input_hdf.attrs["iceLength"]
airHeight = input_hdf.attrs["airHeight"]
dx = input_hdf.attrs["dx"]
dx = input_hdf.attrs["dz"]


Amplitude = input_hdf.attrs["Amplitude"]
freqCentral = input_hdf.attrs["freqCentral"]
freqLP = input_hdf.attrs["freqLP"]
freqHP = input_hdf.attrs["freqHP"]
freqSample = input_hdf.attrs["freqSample"]
freqNyquist = input_hdf.attrs["freqNyquist"]
tCentral = input_hdf.attrs["tCentral"]
tSample = input_hdf.attrs["tSample"]
dt = input_hdf.attrs["dt"]
nSamples = input_hdf.attrs["nSamples"]

tspace = np.linspace(0, dt*nSamples, nSamples)

nRX = input_hdf.attrs["nReceivers"] 
profile_str = input_hdf.attrs["nProf"] 
nTX = input_hdf.attrs["nTransmitters"]


4.4104127999999997e-07
0.00031291480336711623
Snow, eps_r = (1.2+0.015j) n =  (1.0954665093781226+0.006846398256627879j) alpha =  0.2869119187963228
Ice, eps_r = (3.2+0.00031291480336711623j) n = (1.7888543841379774+8.74623462693594e-05j) alpha =  0.003665283357753437


'\nnRX = input_hdf.attrs["nReceivers"] \nprofile_str = input_hdf.attrs["nProf"] \nnTX = input_hdf.attrs["nTransmitters"]\n'

In [3]:
input_hdf.keys()
for tx_key in input_hdf.keys():
    tx = input_hdf[tx_key]
    print(tx)
    sourceDepth = tx.attrs["sourceDepth"]
    for rx_key in tx.keys():
        rx_k = tx[rx_key]
        rx = receiver(rx_k.attrs["range"], rx_k.attrs["depth"])
        print(rx_k.keys())
        print(rx_k.attrs["depth"])
        signal = np.array(rx_k.get('signal'))
        spectrum = np.array(rx_k.get('spectrum'))
        rx.signal = signal
        rx.spectrum = spectrum
        rx.time = tspace
        rx.freq = freq_space
        #print(signal)

<HDF5 group "/tx-1" (1 members)>
<KeysViewHDF5 ['signal', 'spectrum']>
25


NameError: name 'tspace' is not defined

In [ ]:
#Make a Paralell BScan:
Antenna_Seperation = 10.

rx_ranges = np.array(input_hdf.get("rx_ranges"))
tx_depths = np.array(input_hdf.get("tx_depths"))
rx_depths = np.array(input_hdf.get("rx_depths"))

ii_range = util.findNearest(rx_ranges, Antenna_Seperation)
range_selected = rx_ranges[ii_range]

parallel_antenna = []
parallel_signal = np.zeros((nTX, nSamples), dtype='complex')
nFreq_half = int(nSamples/2.)
parallel_spectrum = np.zeros((nTX, nFreq_half), dtype='complex')
ii = 0
for tx_key in input_hdf.keys():
    tx = input_hdf[tx_key]
    print(rxList)
    sourceDepth = tx.attrs["sourceDepth"]
    for rx_key in tx.keys():
        rx_k = tx[rx_key]
        rx = receiver(rx_k.attrs["range"], rx_k.attrs["depth"])
        signal = np.array(rx_k.get('signal'))
        spectrum = np.array(rx_k.get('spectrum'))
        rx.signal = signal
        rx.spectrum = spectrum
        rx.time = tspace
        rx.freq = freq_space
        
        if rx.z == sourceDepth and rx.x == range_selected:
            parallel_signal[ii,:] = rx.get_signal()
            parallel_spectrum[ii,:] = rx.get_spectrum()
            
            ii += 1

In [ ]:
fig = pl.figure(figsize=(10,6),dpi=100)
ax = fig.add_subplot(111)
ax.set_title('Paralell Scan, R = ' + str(range_selected) + ' m')

vmax0 = 0
vmin0 = -100

pmesh = pl.imshow(20*np.log10(abs(parallel_signal)), extent=(tspace[0], tspace[-1], tx_depths[-1], tx_depths[0]), vmin = vmin0, vmax = vmax0, aspect='auto', cmap='hot')
cbar = pl.colorbar(pmesh)
cbar.set_label('Power [dBm]')
ax.set_xlabel('Time [ns]')
ax.set_ylabel('Depth [m]')
pl.show()